# Analyzing geography of FEC contributions

In [1]:
import ast
import copy
import csv

import pandas as pd
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
import requests
import bs4
import us

## open document with challengers info as list of dicts

In [2]:
with open('d_challengers.csv') as f:
     challengers = [{k: v for k, v in row.items()}
          for row in csv.DictReader(f, skipinitialspace=True)]

## set api key, search for candidates entries based on names on FEC API

In [3]:
api_key = "" # <--Set API key here

### name_changes referenced below are fixes for collisions where more than one person would turn up on search results

In [5]:
search_results = []

with open("name_changes.js", "r") as f:
    name_changes = ast.literal_eval(f.read())
f.close()

for ind, row in enumerate(challengers):
    if row["Candidate"] != "" and "?" not in row["Candidate"] and "N/A" not in row["Candidate"]:
        # Place data into dict
        d= {}
        d["name"] = row["Candidate"]
        d["first_name"] = d["name"].split(" ")[0] 
        d["last_name"] = d["name"].split(" ")[-1] 
        d["year"] = "2018"
        d["party"] = "DEM"
        d["office"] = "H"
        d["q2_haul"] = row["Q2 Haul"] 
        for person in name_changes:
            if d["last_name"] == person["orig_last"]:
                try:
                    if d["first_name"] != person["first_name"]:
                        continue
                except:
                    pass
                try:
                    d["year"] = person["year"]
                except:
                    pass
                d["last_name"] = person["new_search"]
                try:
                    d["party"] = person["party"]
                except:
                    pass
        # form URL string for api call
        url = "https://api.open.fec.gov/v1/candidates/search/?q=" + d["last_name"] + "&office=" + d["office"] + "&per_page=100&election_year="+ d["year"] +"&sort=name&page=1&year=" + d["year"] + "&party="+ d["party"] +"&api_key="+ api_key +""
        resp = requests.get(url).text
        resp_clean = resp.replace("null",'""').replace("\n","").replace(':true',':"true"').replace(':false',':"false"')
        api_dict = ast.literal_eval(resp_clean)
        api_dict["orig_data"] = copy.copy(d)
        # fixes instance where there candidate's name is part of other candidate's name
        if "SHAUN BROWN" in str(api_dict):
            api_dict["results"] = [copy.deepcopy(api_dict["results"][0])]
        search_results.append(api_dict)

## print those where the number of results != 1 to see where there may be problems

In [6]:
for result in search_results:
    if (len(result["results"]) != 1) and ((len(result["results"]) !=0) and (result["orig_data"]["q2_haul"] != 0 or result["orig_data"]["q2_haul"] != "")):
        print(len(result["results"]))
        print("\n")
        print(str(result["orig_data"]) + "\n")
        for item in result["results"]:
            print(item)
            print("\n")
        print("\n---------\n\n")
    if (len(result["results"]) != 1):
        print(str(result["orig_data"]))
        print("---------")

{'name': 'Kremer', 'first_name': 'Kremer', 'last_name': 'skip', 'year': '2018', 'party': 'DEM', 'office': 'H', 'q2_haul': ''}
---------
{'name': 'Sigala', 'first_name': 'Sigala', 'last_name': 'skip', 'year': '2018', 'party': 'DEM', 'office': 'H', 'q2_haul': ''}
---------
{'name': 'Debbie Mucarsel-Powell', 'first_name': 'Debbie', 'last_name': 'Mucarsel-Powell', 'year': '2018', 'party': 'DEM', 'office': 'H', 'q2_haul': ''}
---------


## define function to get committee contribution data

In [7]:
def get_data(committee_id):
    url = "https://api.open.fec.gov/v1/schedules/schedule_a/?api_key="+ api_key +"&committee_id=" + committee_id + "&two_year_transaction_period=2018&per_page=100&sort=contribution_receipt_date"
    orig_url = copy.copy(url)

    resp = requests.get(url)

    # modifies response to make it python dict compatiable
    resp_text = resp.text.replace("null",'""').replace("\n","").replace(':true',':"true"').replace(':false',':"false"')

    # evaluates response into dict
    resp_dict = ast.literal_eval(resp_text)

    # marks that we are not yet past last page of api results
    empty_page = 0

    # sets initial list of member contributions equal to first response from api
    member_contributions = resp_dict["results"]


    # until you get past all pages of api response
    while empty_page == 0:
    #     print(resp_dict)
        #adds parameters for last index and last contrib date to new url
        url = orig_url + "&last_index=" + str(resp_dict["pagination"]["last_indexes"]["last_index"]) + "&last_contribution_receipt_date=" + resp_dict["pagination"]["last_indexes"]["last_contribution_receipt_date"]
        # modifies response to make it python dict compatiable
        resp_text = requests.get(url).text.replace("null",'""').replace("\n","").replace(':true',':"true"').replace(':false',':"false"')
        # evaluates response into dict
        resp_dict = resp_dict = ast.literal_eval(resp_text)
        # marks we are past all pages of api response if no results     
        if len(resp_dict["results"]) == 0:
            empty_page = 1
        # if there are results, appends them to list of member contribs  
        else:
            member_contributions = copy.deepcopy(member_contributions) + resp_dict["results"]

    
    return(member_contributions)



## place data into list of dict that has each candidate's info, and fails to parse those without FEC processed contribs

In [8]:
search_results_with_data = copy.deepcopy(search_results)

for result in search_results_with_data:
    result["committees_info"] = []
    try:
        for committee in result["results"][0]["principal_committees"]:
            if 2018 in committee["cycles"]:
                d = copy.copy(committee)
                d["committee_data"] = get_data(committee["committee_id"])
                result["committees_info"].append(d)
    except:
        print("could not parse " + result["orig_data"]["name"])

could not parse Kirkpatrick
could not parse Verdin
could not parse Cox
could not parse Bliatout
could not parse Franco
could not parse Kremer
could not parse Sigala
could not parse Gilbert Cisneros
could not parse Sam Jammal
could not parse Andy Thorburn
could not parse Brian Forde
could not parse Michael Kotick
could not parse Boyd Roberts
could not parse Omar Siddiqui
could not parse Ammar Campa
could not parse Diane Mitsch Bush
could not parse Levi Tillemann
could not parse Nancy Soderberg
could not parse Pam Keith
could not parse Debbie Mucarsel-Powell


## creates new list of dicts from search results with easier to access relevant info, including candidate data and committee data/contribs, and prints names of those without neded info

In [111]:
# want this to exclude ones with no data

cand_data = []

for result in search_results_with_data:
    d = {}
    try:
        d["state"] = result["results"][0]["state"]
        d["district"] = result["results"][0]["district"]
        d["party"] = result["results"][0]["party"]
        d["committees_info"] = result["committees_info"]        
        d["last_name"] = result["orig_data"]["last_name"]
        d["committee_ids"] = []
        d["candidate_name"] = result["results"][0]["name"]
        for com in result["results"][0]["principal_committees"]:            
            d["committee_ids"].append(com["committee_id"])
        d["candidate_id"] = result["results"][0]["candidate_id"]
        if len(d["committees_info"]) != 0:
            cand_data.append(d) 
        else:
            print(d["last_name"])
    except:
        print(result["orig_data"]["name"])
        continue        

Kirkpatrick
Verdin
Cox
Bliatout
Franco
Kremer
Sigala
Cisneros
Jammal
Thorburn
Forde
Kotick
Roberts
Siddiqui
MITSCH BUSH, DIANE MS.
Tillemann
Soderberg
KEITH, PAM
Debbie Mucarsel-Powell


## creates all_receipts list of dicts which has every receipt included

In [12]:
all_receipts = []

for cand in cand_data:
    cand["total_receipts"] = 0    
    for com in cand["committees_info"]:
        for receipt in com["committee_data"]:
            d = {}
            d["candidate_name"] = cand["candidate_name"]
            d["committee_state"] = com["state"]
            d["candidate_state"] = cand["state"]
            d["candidate_dist"] = cand["district"]
            d["amendment_indicator_desc"] = receipt["amendment_indicator_desc"]
            d["committee_id"] = receipt["committee"]["committee_id"]
            d["cycle"] = str(receipt["committee"]["cycle"])
            d["designation_full"] = receipt["committee"]["designation_full"]
            d["committee_name"] = receipt["committee"]["name"]
            d["amount"] = receipt["contribution_receipt_amount"]
            d["contribution_receipt_date"] = receipt["contribution_receipt_date"]
            d["contributor"] = receipt["contributor"]
            d["contributor_aggregate_ytd"] = receipt["contributor_aggregate_ytd"]
            d["contributor_city"] = receipt["contributor_city"]
            d["contributor_employer"] = receipt["contributor_employer"]
            d["contributor_first_name"] = receipt["contributor_first_name"]
            d["contributor_last_name"] = receipt["contributor_last_name"]
            d["contributor_middle_name"] = receipt["contributor_middle_name"]
            d["contributor_id"] = receipt["contributor_id"]
            d["contributor_occupation"] = receipt["contributor_occupation"]
            d["contributor_state"] = receipt["contributor_state"]
            d["contributor_zip"] = receipt["contributor_zip"]
            d["entity_type_desc"] = receipt["entity_type_desc"]
            d["fec_election_type_desc"] = receipt["fec_election_type_desc"]
            d["is_individual"] = receipt["is_individual"]
            d["memoed_subtotal"] = receipt["memoed_subtotal"]
            d["memo_text"] = receipt["memo_text"]
            d["receipt_type"] = receipt["receipt_type"]
            d["receipt_type_full"] = receipt["receipt_type_full"]
            d["report_type"] = receipt["report_type"]
            d["report_year"] = str(receipt["report_year"])
            d["transaction_id"] = receipt["transaction_id"]
            d["two_year_transaction_period"] = str(receipt["two_year_transaction_period"])
            all_receipts.append(d)
            

## converts ZIPs to ZCTs, printing status every 500 and numbers/states for ZIPs that do not properly convert

In [13]:
with open('zip_to_zcta_2017.csv') as f:
     zip_zcta = [{k: v for k, v in row.items()}
          for row in csv.DictReader(f, skipinitialspace=True)]

print("total receipts:" + str(len(all_receipts)))        

for ind,receipt in enumerate(all_receipts):
    if ind%500 ==0:
        print("starting " + str(ind))
    receipt["ZCTA"] = ""
    if receipt["contributor_zip"] != "":
        try:
            receipt["ZCTA"] = [d for d in zip_zcta if (d["ZIP_CODE"] == receipt["contributor_zip"][0:5])][0]["ZCTA"]
        except:
            print(receipt["contributor_zip"] + "" + receipt["contributor_state"])

total receipts:24133
starting 0
starting 500
00000ZZ
starting 1000
200760003DC
starting 1500
90365CA
90365CA
starting 2000
starting 2500
starting 3000
00000ZZ
starting 3500
starting 4000
starting 4500
starting 5000
090460006AE
starting 5500
starting 6000
91002CA
6831CT
6831CT
82075CA
91002CA
82075CA
starting 6500
starting 7000
starting 7500
starting 8000
00000ZZ
starting 8500
starting 9000
00000FL
starting 9500
starting 10000
starting 10500
starting 11000
starting 11500
starting 12000
00000ZZ
starting 12500
00000ZZ
962043026AP
starting 13000
starting 13500
01142
01142
starting 14000
starting 14500
starting 15000
starting 15500
starting 16000
starting 16500
starting 17000
starting 17500
starting 18000
starting 18500
starting 19000
starting 19500
H3G1J4ZZ
starting 20000
starting 20500
starting 21000
00000ZZ
098469997AE
00000ZZ
starting 21500
starting 22000
starting 22500
starting 23000
00000AA
starting 23500
67000
00075
09213AE
starting 24000
98705WA


# get database of CDs and zip codes crossovers from Census Bureau

In [14]:
census_zip_pg = requests.get("https://www.census.gov/geo/maps-data/data/cd_state.html").text

soup_census_zip_pg = bs4.BeautifulSoup(census_zip_pg, "html.parser")

state_menu = soup_census_zip_pg.find_all("select", {"id": "zc113Stated"})

states_113 = []

for option in state_menu[0].find_all("option"):
    if "Select a State" not in option:
        d = {}
        d["state"] = option.get_text()
        d["url"] = option["value"]
        states_113.append(d)
        
def parse_urls(states, destination_location):
    for state in states:
    #     get data for each
        zip_data = requests.get(state["url"]).text.split("\n")[1:]
    #     clean extra characters
        for ind, row in enumerate(zip_data):
            zip_data[ind] = row.replace("\r", "" )
    #     set headers
        headers = []
        for item in zip_data[0].split(","):
            headers.append(item)
    #     put each row of data in a dict and append to larger dict
        for ind, row in enumerate(zip_data):
            if ind != 0 and len(row.split(",")) >= 2:
                d= {}
                for ind2, item in enumerate(row.split(",")):
                    d[headers[ind2]] = item
                d["state_name"] = state["state"]
                destination_location.append(d)
                
# get 113th congress data in list of dicts

all_zip_rows = []

parse_urls(states_113, all_zip_rows)    


# get updated CD/zips

soup_115 = bs4.BeautifulSoup(census_zip_pg.split("Congressional Districts by ZIP Code Tabulation Areas")[1], "html.parser")
zip_links = soup_115.find_all("div", {"class": "indent"})[0].find_all("a")
states_115 = []
for ind, link in enumerate(zip_links):
    if ind >= len(zip_links)/2:
        d= {}
        d["state"] = link.get_text()
        d["url"] = link["href"]
        states_115.append(d)

        
# get 113th congress data in list of dicts

all_rows_115 = []

parse_urls(states_115, all_rows_115)    


# replace old 113th state zips with updated 115th state zips

new_zip_list = copy.deepcopy(all_rows_115)

# identifies which states got changed
changed_state_codes = []

for row in all_rows_115:
    if row["State"] not in changed_state_codes:
        changed_state_codes.append(row["State"])

#  go through old rows and add to new_zip_list if not one of modified states
for row in all_zip_rows:
    if row["State"] not in changed_state_codes:
        new_zip_list.append(row)

## compare all receipts to ZIP codes to see if contrib is in district, and prints out states of receipts that cannot be parsed

In [15]:
receipts_missing_zips = []
receipts_unknown_CD = []

for receipt in all_receipts:
    receipt["CD"] = []
    receipt["in_CD"] = "no"
    if receipt["contributor_state"] == "ZZ" or receipt["contributor_state"] == "AP" or receipt["contributor_state"] == "AA" or receipt["contributor_state"] == "AE":
        receipt["CD"] = ["Out of country/armed forces"]
        receipt["in_CD"] = "no"
        continue
    if receipt["contributor_state"] == "FM":
        receipt["CD"] = ["none"]
        receipt["in_CD"] = "no"
        continue
    if receipt["contributor_state"] == "DC":
        receipt["CD"] = ["none"]
        receipt["in_CD"] = "no"
        continue
    try:
        long_contrib_state = us.states.lookup(receipt["contributor_state"]).name
    except:
        print("could not parse state abbrv: '" + receipt["contributor_state"] + "'")
        continue
    state_zip_data = [d for d in new_zip_list if d['state_name'] == long_contrib_state]
    if len(state_zip_data) == 0:
        receipt["CD"] = ["At large"]
        if long_contrib_state == receipt["candidate_state"] or long_contrib_state == us.states.lookup(receipt["candidate_state"]).name:
            receipt["in_CD"] = "yes"
        continue
    if receipt["contributor_zip"] == "":
        receipt["in_CD"] = "missing"
        receipts_missing_zips.append(receipt) 
        continue
    found_zip = False
    for row in state_zip_data:
        if str(receipt["ZCTA"]) == row["ZCTA"]:
            found_zip = True
        if str(receipt["ZCTA"]) == row["ZCTA"] and receipt["contributor_state"] == receipt["candidate_state"]:
            receipt["CD"].append(row["CongressionalDistrict"])            
            if int(receipt["candidate_dist"]) == int(row["CongressionalDistrict"]):
                receipt["in_CD"] = "yes"                  
    if found_zip == False:
        receipt["in_CD"] = "unknown"
        receipts_unknown_CD.append(receipt)
            

could not parse state abbrv: ''
could not parse state abbrv: ''
could not parse state abbrv: ''
could not parse state abbrv: ''
could not parse state abbrv: ''
could not parse state abbrv: ''
could not parse state abbrv: ''
could not parse state abbrv: ''
could not parse state abbrv: ''
could not parse state abbrv: ''
could not parse state abbrv: ''


## Check to see which receipts did not get properly categorized into a congressional district

In [16]:
missing_amts = []

for el in receipts_missing_zips:
    d = {}
    d["amount"] = el["amount"]
    d["name"] = el["candidate_name"]
    missing_amts.append(d)

missing_amts

[{'amount': 50.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 100.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 50.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 100.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 100.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 100.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 50.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 1000.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 100.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 50.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 100.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 100.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 50.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 100.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 50.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 2700.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 700.0, 'name': 'APPLEGATE, DOUGLAS LOREN'},
 {'amount': 1000.0, 'name': 'LEVIN, MIKE'},
 {'amoun

# filters down to individual itemized receipts ONLY, and puts into pandas df
## creates same_state col in df that indicates whether contrib came from in candidate's state

In [76]:
receipts_df = pd.DataFrame(all_receipts)

indv_receipts_df = receipts_df[((receipts_df["is_individual"] == "true"))]
indv_receipts_df["same_state"] = (indv_receipts_df["candidate_state"] == indv_receipts_df["contributor_state"])

/Users/smcminn/Documents/graphics/dem_q2_fec/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


## gets ONLY contributions for each candidate that came from out of dist

In [75]:
indv_receipts_df[indv_receipts_df["in_CD"] == "no"].groupby(["candidate_name", "candidate_state", "candidate_dist"]).sum().sort_values("amount", ascending=0)

amount  \
candidate_name              candidate_state candidate_dist              
DELGADO, ANTONIO            NY              19              542110.00   
HARDER, JOSH                CA              10              384695.00   
LEVIN, MIKE                 CA              49              377164.08   
FRIEDMAN, ALISON            VA              10              360796.99   
PHILLIPS, DEAN              MN              03              313605.00   
HELMER, DANIEL              VA              10              289384.51   
HUFFSTETLER, ROGER DEAN RD  VA              05              272504.45   
TRAN, MAI-KHANH             CA              39              244080.00   
STOVER, LINDSEY DAVIS       VA              10              242315.42   
MCCREADY, DANIEL            NC              09              233974.45   
FLYNN, BRIAN                NY              19              227904.34   
PORTER, KATHERINE           CA              45              225927.80   
MIN, DAVID                  CA              45              225297.00   
TRIANTAPHYLLIS, ALEXANDER   TX              07              223883.00   
HOULAHAN, CHRISSY           PA              06              223770.00   
MEIER, EDWARD               TX              32              213615.37   
STEVENS, HALEY              MI              11              210260.23   
CROW, JASON                 CO              06              196316.88   
RYAN, PATRICK               NY              19              162195.44   
CAFORIO, BRYAN              CA              25              149768.37   
MOSER, LAURA                TX              07              142669.00   
HILL, KATHERINE LAUREN      CA              25              128788.00   
APPLEGATE, DOUGLAS LOREN    CA              49              126189.34   
HARBAUGH, KENNETH           OH              07              120817.99   
HEINZ, MATTHEW GERALD       AZ              02              119450.00   
CAMPA-NAJJAR, AMMAR         CA              50              117437.82   
FLETCHER, ELIZABETH PANNILL TX              07              113250.00   
HAMADANCHY, KIA             CA              45              107855.49   
FINKENAUER, ABBY            IA              01              106057.37   
ROUDA, HARLEY E  JR         CA              48              100379.00   
RHODES, GARETH THOMAS       NY              19               99768.00   
ROSEN GONZALEZ, KRISTEN     FL              27               95617.00   
MUROFF, DANIEL              PA              07               95118.00   
KIM, DAVID                  GA              07               95060.00   
SHERRILL, REBECCA MICHELLE  NJ              11               91904.68   
BUTNER, JOSH                CA              50               79391.45   
VARGAS, JUAN C.             CA              51               78350.00   
WESTIN, JASON               TX              07               76996.00   
JANOWICZ, PHIL ANDREW DR.   CA              39               72813.00   
RAMSEY, ANDREA              KS              03               71525.00   
SRAN, DEEP                  VA              10               65779.00   
KIM, ANDY                   NJ              03               63608.52   
RADER, ELIZABETH ANNE       OH              14               59960.00   
KEIRSTEAD, HANS             CA              48               57725.00   
WEXTON, JENNIFER            VA              10               56899.73   
ALLRED, COLIN               TX              32               55562.00   
MAZESKI, KELLY              IL              06               55560.00   
WEBER, LINDA                NJ              07               51101.00   
MORGAN, MATTHEW WADE        MI              01               42930.00   
MANDELBLATT, LISA           NJ              07               42318.00   
KERR, ANDREW STEVEN         CO              07               41501.48   
MELSON, BOYD                NY              11               39017.15   
PHOENIX, JESS               CA              25               37364.00   
SALMON, SCOTT               NJ              07  

## func to pull this year's reported totals to compare itemized indv totals

In [117]:
def get_recent_totals(committee_id): 
    
    d = {}
    
    d["individual_itemized_contributions"] = 0
    d["individual_contributions"] = 0
    d["individual_unitemized_contributions"] = 0
    d["contributions"] = 0
    

    url = "https://api.open.fec.gov/v1/committee/"+ committee_id +"/totals/?sort=-cycle&api_key="+ api_key +"&per_page=100&page=1"
    orig_url = copy.copy(url)

    resp = requests.get(url)

    # modifies response to make it python dict compatiable
    resp_text = resp.text.replace("null",'""').replace("\n","").replace(':true',':"true"').replace(':false',':"false"')

    # evaluates response into dict
    try:
        resp_dict = ast.literal_eval(resp_text)
    except:
        print("ERROR ABOVE RESP TEXT PARSING")

    # marks that we are not yet past last page of api results
    empty_page = 0

    # sets initial list of totals equal to first response from api
    try:
        totals = resp_dict["results"]
    except:
        print("NO TOTALS FOR " + committee_id)

    # logs number of pgs expected
    pagination_count = resp_dict["pagination"]["count"]

    # until you get past all pages of api response
    while empty_page == 0:
        #adds parameters for last index and last contrib date to new url
        url = url.split("&page=")[0] + "&page=" + str(int(url.split("&page=")[1]) + 1)
        # modifies response to make it python dict compatiable
        resp_text = requests.get(url).text.replace("null",'""').replace("\n","").replace(':true',':"true"').replace(':false',':"false"')
        # evaluates response into dict
        resp_dict = ast.literal_eval(resp_text)
        # marks we are past all pages of api response if no results     
        if len(resp_dict["results"]) == 0:
            empty_page = 1
        # if there are results, appends them to list of member contribs  
        else:
            totals = copy.deepcopy(totals) + resp_dict["results"]
            time.sleep(5)


    if pagination_count - len(totals) > 1:
        print("count error for " + committee_id)        

    for total in totals:        
        if total["cycle"] == 2018:
            d["individual_itemized_contributions"] = d["individual_itemized_contributions"] + total["individual_itemized_contributions"]
            d["individual_contributions"] = d["individual_contributions"] + total["individual_contributions"]
            d["individual_unitemized_contributions"] = d["individual_unitemized_contributions"] + total["individual_unitemized_contributions"]
            d["contributions"] = d["contributions"] + total["contributions"]
            break

    d["committee_id"] = committee_id
#     d["committee_name"] = str(d["committee_name"] + " " + total["committee_name"]).strip()

    
    return(d)

## gets filing report for each candidate

In [118]:
cand_totals = []

for cand in cand_data:
    d = {}
    d["contributions"] = 0
    d['individual_contributions']= 0
    d['individual_itemized_contributions'] = 0
    d['individual_unitemized_contributions'] =  0
    for com in cand["committee_ids"]:
        temp_d = copy.copy(get_recent_totals(com))
        d["contributions"] = d["contributions"] + temp_d["contributions"]
        d['individual_contributions'] = d['individual_contributions'] + temp_d["individual_contributions"]
        d['individual_itemized_contributions'] = d['individual_itemized_contributions'] + temp_d["individual_itemized_contributions"]
        d['individual_unitemized_contributions'] = d['individual_unitemized_contributions'] + temp_d["individual_unitemized_contributions"]
    d["candidate_name"] = cand["candidate_name"]
    d["committees"] = cand["committee_ids"]
    cand_totals.append(d)

## makes df out of candidate totals and compares to itemized receipts (in "diff" col)

In [119]:
cand_totals_df = pd.DataFrame(cand_totals)
candidate_receipts_df  = indv_receipts_df.groupby(["candidate_name"]).sum().sort_values("amount", ascending=0)
candidate_receipts_df.reset_index(level=0, inplace=True)

total_check_merge = pd.merge(left=cand_totals_df, right=candidate_receipts_df, on="candidate_name")
total_check_merge["diff"] = total_check_merge["individual_itemized_contributions"] - total_check_merge["amount"]

total_check_merge[total_check_merge["diff"] != 0].sort_values("diff", ascending=0)

candidate_name  \
25      APPLEGATE, DOUGLAS LOREN   
22           ROUDA, HARLEY E  JR   
52                PHILLIPS, DEAN   
34       ROSEN GONZALEZ, KRISTEN   
21             PORTER, KATHERINE   
100               HELMER, DANIEL   
90                 ALLRED, COLIN   
80                MUROFF, DANIEL   
64                  MELSON, BOYD   
83     TRIANTAPHYLLIS, ALEXANDER   
86   FLETCHER, ELIZABETH PANNILL   
62    SHERRILL, REBECCA MICHELLE   
13               VARGAS, JUAN C.   
2        KOVACS, WILLIAM RICHARD   
102   MCDEVITT, DAVID MALCOLM MR   
61                 SALMON, SCOTT   
70                 SULLIVAN, SUE   
20                    MIN, DAVID   
75             HARBAUGH, KENNETH   
68         RHODES, GARETH THOMAS   
56                     KIM, ANDY   
93           BROWN, SHAUN DENISE   
96         STOVER, LINDSEY DAVIS   
76         RADER, ELIZABETH ANNE   
37                   AXNE, CINDY   
48          MORGAN, MATTHEW WADE   
82                 KING, JESSICA   
72             BRINDISI, ANTHONY   
3          HEINZ, MATTHEW GERALD   
58                  WEBER, LINDA   
85               CARGAS, JAMES P   
17     JANOWICZ, PHIL ANDREW DR.   
14                CAFORIO, BRYAN   
50                STEVENS, HALEY   
51                 ERDMANN, JEFF   
71             NELSON, PATRICK F   
97              WEXTON, JENNIFER   
69                 RYAN, PATRICK   
8        NYGARD, DOROTHY (DOTTY)   
59             MANDELBLATT, LISA   
30           CAMPA-NAJJAR, AMMAR   
15        HILL, KATHERINE LAUREN   
27                   LEVIN, MIKE   
23               KEIRSTEAD, HANS   
94    HUFFSTETLER, ROGER DEAN RD   

                                       committees  contributions  \
25                                    [C00581595]      397491.60   
22                                    [C00633982]      319333.90   
52                                    [C00640714]      533986.11   
34                                    [C00637546]      184411.42   
21                                    [C00636571]      311548.70   
100                                   [C00636738]      368621.10   
90                                    [C00637868]      180591.04   
80                                    [C00636563]      149510.23   
64                                    [C00637587]       56791.19   
83                                    [C00639187]      451103.53   
86                                    [C00640045]      365721.00   
62                                    [C00640003]      245957.25   
13   [C00497321, C00408161, C00307256, C00259705]      235318.70   
2                                     [C00636175]       20260.15   
102                                   [C00592345]       11887.93   
61                                    [C00637728]       53189.78   
70                                    [C00640680]       54519.77   
20                                    [C00636845]      304206.91   
75                                    [C00646752]      191478.11   
68                                    [C00639559]      134962.78   
56                                    [C00648220]      117236.45   
93                                    [C00615013]       38517.59   
96                                    [C00637892]      306651.34   
76                                    [C00637967]      112166.19   
37                                    [C00646844]       81147.00   
48                                    [C00636324]      115578.08   
82                                    [C00648774]       12660.64   
72                                    [C00648725]       40974.00   
3                                     [C00647727]      201086.00   
58                                    [C00637090]      122201.36   
85                                    [C00509299]       35708.39   
17                                    [C00638478]      104284.24   
14                                    [C00597724]      212963.66   
50                                    [C00638650]

## get in, out CD/state breakdowns for candidates

In [23]:
all_indv_breakdowns_df = indv_receipts_df.groupby(["candidate_name", "candidate_state", "candidate_dist", "same_state", "in_CD"]).sum().reset_index()

indv_totals_df = indv_receipts_df.groupby(["candidate_name", "candidate_state", "candidate_dist"]).sum().reset_index()

categories = [{"name": "in_CD", "conditions": [{"col": "in_CD", "result": "yes"}]}, {"name": "out_CD_in_state", "conditions": [{"col": "in_CD", "result": "no"}, {"col": "same_state", "result": True}]}, {"name": "out_state", "conditions": [{"col": "same_state", "result": False}]}, {"name": "missing", "conditions": [{"col": "in_CD", "result": "missing"}]}]

indv_totals_df_merge = pd.DataFrame()

# only works if first item in categories has only ONE condition in list of conditions
for ind, category in enumerate(categories):
    if ind == 0:
        indv_totals_df_merge = pd.merge(indv_totals_df, all_indv_breakdowns_df[all_indv_breakdowns_df[category["conditions"][0]["col"]] == category["conditions"][0]["result"]][["candidate_name", "amount"]].rename(index=str, columns={"amount": "amount_" + category["name"]}), how="left", on="candidate_name").fillna(0)
    if ind >= 1:
        merge_right = all_indv_breakdowns_df[all_indv_breakdowns_df[category["conditions"][0]["col"]] == category["conditions"][0]["result"]].rename(index=str, columns={"amount": "amount_" + category["name"]})
        condition_cols = [category["conditions"][0]["col"], "candidate_name"]
        if len(category["conditions"]) > 1:            
            for condition in category["conditions"][1:]:
                merge_right = merge_right[merge_right[condition["col"]] == condition["result"]]
                condition_cols.append(condition["col"])
        merge_right = merge_right.groupby(condition_cols).sum().reset_index()
        merge_right = merge_right[["candidate_name", "amount_" + category["name"]]]        
        indv_totals_df_merge = pd.merge(indv_totals_df_merge, merge_right, how="left", on="candidate_name").fillna(0)

indv_totals_df_merge["amount_sum"] = indv_totals_df_merge["amount_in_CD"] + indv_totals_df_merge["amount_out_CD_in_state"] + indv_totals_df_merge["amount_out_state"] + indv_totals_df_merge["amount_missing"] 
indv_totals_df_merge["amount_check"] = indv_totals_df_merge["amount_sum"] - indv_totals_df_merge["amount"]

for category in categories:
    indv_totals_df_merge["amount_" + category["name"] + "_pct"] = indv_totals_df_merge["amount_" + category["name"]] * 100 / indv_totals_df_merge["amount"]

## df of candidates with amounts and pcts of in/out CD/state contributions

In [24]:
indv_totals_df_merge.sort_values("amount", ascending=0)

candidate_name candidate_state candidate_dist     amount  \
17              DELGADO, ANTONIO              NY             19  601470.00   
71                PHILLIPS, DEAN              MN             03  495605.00   
51                   LEVIN, MIKE              CA             49  435420.24   
56              MCCREADY, DANIEL              NC             09  421291.18   
96     TRIANTAPHYLLIS, ALEXANDER              TX             07  391881.00   
25              FRIEDMAN, ALISON              VA             10  389597.99   
31                  HARDER, JOSH              CA             10  385695.00   
23   FLETCHER, ELIZABETH PANNILL              TX             07  340007.00   
34                HELMER, DANIEL              VA             10  322139.34   
37             HOULAHAN, CHRISSY              PA             06  316225.00   
40    HUFFSTETLER, ROGER DEAN RD              VA             05  313289.45   
60                 MEIER, EDWARD              TX             32  308037.37   
91         STOVER, LINDSEY DAVIS              VA             10  274990.42   
2       APPLEGATE, DOUGLAS LOREN              CA             49  263080.34   
62                    MIN, DAVID              CA             45  262777.00   
90                STEVENS, HALEY              MI             11  260251.67   
95               TRAN, MAI-KHANH              CA             39  251980.00   
73             PORTER, KATHERINE              CA             45  246472.80   
24                  FLYNN, BRIAN              NY             19  234604.34   
16                   CROW, JASON              CO             06  228791.88   
80           ROUDA, HARLEY E  JR              CA             48  225910.00   
65                  MOSER, LAURA              TX             07  195892.08   
82                 RYAN, PATRICK              NY             19  185395.44   
79       ROSEN GONZALEZ, KRISTEN              FL             27  178067.00   
33         HEINZ, MATTHEW GERALD              AZ             02  166275.00   
13                CAFORIO, BRYAN              CA             25  163755.40   
28               HAMADANCHY, KIA              CA             45  163150.49   
85    SHERRILL, REBECCA MICHELLE              NJ             11  158890.05   
76                RAMSEY, ANDREA              KS             03  154300.00   
36        HILL, KATHERINE LAUREN              CA             25  152122.00   
101             WEXTON, JENNIFER              VA             10  148460.87   
30             HARBAUGH, KENNETH              OH             07  142348.99   
14           CAMPA-NAJJAR, AMMAR              CA             50  137084.82   
52             MANDELBLATT, LISA              NJ             07  134234.81   
1                  ALLRED, COLIN              TX             32  132440.00   
44               KEIRSTEAD, HANS              CA             48  128725.00   
22              FINKENAUER, ABBY              IA             01  125198.37   
100                WESTIN, JASON              TX             07  120911.00   
66                MUROFF, DANIEL              PA             07  118818.00   
48                    KIM, DAVID              GA             07  118210.00   
12                  BUTNER, JOSH              CA             50  107696.45   
77         RHODES, GARETH THOMAS              NY             19  107168.00   
99                  WEBER, LINDA              NJ             07   99869.74   
55                MAZESKI, KELLY              IL             06   94946.41   
69                 OATMAN, LAURA              CA             48   90998.41   
97               VARGAS, JUAN C.              CA             51   90250.00   
75         RADER, ELIZABETH ANNE              OH             14   85529.61   
42     JANOWICZ, PHIL ANDREW DR.              CA             39   85354.48   
63          MORGAN, MATTHEW WADE              MI             01   77948.14   
89                    SRAN, DEEP              VA             10   74529.00   
..                           ...        

## make df of individuals who have raised more than 50k with less than 10 percent from in district.

In [120]:
big_earners_df = indv_totals_df_merge[["amount", "candidate_name","candidate_state","candidate_dist","amount", "amount_in_CD_pct", "amount_out_CD_in_state_pct", "amount_out_state_pct", "amount_missing_pct"]][indv_totals_df_merge["amount"] > 50000].sort_values("amount_in_CD_pct", ascending=1)

## Export CSV for individual zip level data

In [27]:
cand_receipts_df = pd.DataFrame(all_receipts)

In [77]:
cand_receipts_df["zip_first_five"] = cand_receipts_df["contributor_zip"].str[0:5]
cand_receipts_df[(cand_receipts_df["is_individual"]== "true")].groupby("zip_first_five").sum()["amount"].reset_index().to_csv("cand_zip.csv" , index=False)

## look at unknown CD receipts

In [78]:
unknown_df_indv = pd.DataFrame([d for d in receipts_unknown_CD if (d["is_individual"] == "true")])
unknown_df_indv["zip_first_five"] = unknown_df_indv["contributor_zip"].str[0:5]
unknown_df_indv.groupby(["zip_first_five", "contributor_state"]).sum().sort_values("amount", ascending= 0)

amount  contributor_aggregate_ytd
zip_first_five contributor_state                                   
33921          CO                 6900.0                     8400.0
33134          LA                 2500.0                     2500.0
00000          FL                 1000.0                     1000.0
90365          CA                 1000.0                     1500.0
80228          CA                 1000.0                     1000.0
98705          WA                 1000.0                     1000.0
53589          WA                  500.0                      500.0
10960          NJ                  500.0                      500.0
06468          CA                  500.0                      500.0
60302          ID                  500.0                      500.0
98033          PA                  475.0                     1225.0
91002          CA                  350.0                      700.0
28210          GA                  250.0                      250.0
90035          VA                  250.0                      250.0
01201          PA                  200.0                      250.0
6831           CT                  200.0                      400.0
82075          CA                  200.0                      900.0

## categorize into regions filings for contribs to candidates that got 90% out of district and raised more than $50k

In [72]:
big_earners_low_cd_names = list(big_earners_df[big_earners_df["amount_in_CD_pct"] <=10]["candidate_name"])

In [73]:
big_earners_low_cd_receipts = [d for d in all_receipts if (d["candidate_name"] in big_earners_low_cd_names)]

In [32]:
# state regions from census regional distinctions, found here https://github.com/cphalpert/census-regions/blob/master/us%20census%20bureau%20regions%20and%20divisions.csv

with open('cenus-regions.csv') as f:
     state_region_dicts = [{k: v for k, v in row.items()}
          for row in csv.DictReader(f, skipinitialspace=True)]

In [33]:
no_region_receipts = []

for receipt in big_earners_low_cd_receipts:
    receipt["chart_disposition"] = ""
    receipt["contributor_region"] = ""
    if receipt["in_CD"] == "yes":
        receipt["chart_disposition"] = "in_CD"
        continue
    if receipt["contributor_state"] != receipt["candidate_state"]:
        receipt["chart_disposition"] = "out_of_state"
        try:
            receipt["contributor_region"] = [d for d in state_region_dicts if (d["State Code"] == receipt["contributor_state"])][0]["Region"]
        except:
            no_region_receipts.append(receipt)
            receipt["contributor_region"] = "other"
        continue
    else:
        receipt["chart_disposition"] = "in_state_out_CD"        
        
big_earners_low_cd_receipts_indv = [d for d in big_earners_low_cd_receipts if (d["is_individual"] == "true")]

## get region breakdown for each candidate in big earner low in CD

In [84]:
census_region_df = pd.read_csv('cenus-regions.csv')

indv_receipts_region_merge_df = pd.merge(indv_receipts_df, census_region_df, left_on="contributor_state", right_on="State Code")

In [122]:
indv_receipts_region_merge_df[(indv_receipts_region_merge_df["candidate_name"].isin(big_earners_low_cd_names)) & (indv_receipts_region_merge_df["same_state"] == 0)].groupby(["candidate_name", "Region"]).sum()

amount  contributor_aggregate_ytd  \
candidate_name             Region                                            
CAFORIO, BRYAN             Midwest       500.00                     500.00   
                           Northeast   11450.00                   11450.00   
                           South       22350.00                   27925.00   
                           West         5335.00                    8285.00   
DELGADO, ANTONIO           Midwest      6200.00                    7950.00   
                           Northeast   79355.00                  114355.00   
                           South       72125.00                   96788.00   
                           West        88445.00                  117795.00   
FLYNN, BRIAN               Midwest      5650.00                   11050.00   
                           Northeast   63525.00                  120675.00   
                           South       25475.00                   53075.00   
                           West         1100.00                    1350.00   
FRIEDMAN, ALISON           Midwest     11850.00                   11850.00   
                           Northeast   46950.00                   48200.00   
                           South       92100.00                  127800.00   
                           West       202996.99                  286746.99   
HARDER, JOSH               Midwest      2500.00                    2500.00   
                           Northeast  214950.00                  380400.00   
                           South       21950.00                   22500.00   
                           West         7650.00                   13050.00   
HELMER, DANIEL             Midwest      6985.00                    9310.00   
                           Northeast  138700.00                  169443.16   
                           South       74150.30                   80475.30   
                           West        52074.21                   61254.21   
HUFFSTETLER, ROGER DEAN RD Midwest      1500.00                    2500.00   
                           Northeast   98519.45                  149469.45   
                           South       53930.00                   84460.00   
                           West       117755.00                  208615.00   
KIM, ANDY                  Midwest      1250.00                    1250.00   
                           Northeast   11914.75                   12107.07   
                           South       22266.33                   33612.35   
                           West        18351.79                   30724.49   
PORTER, KATHERINE          Midwest     21350.00                   23748.00   
                           Northeast   55450.00                   68300.00   
                           South       48905.55                   66055.55   
                           West         8565.00                    9380.00   
RHODES, GARETH THOMAS      Midwest       500.00                     500.00   
                           Northeast    4930.00                    5919.00   
                           South        7750.00                    9850.00   
                           West          750.00                     800.00   
TRAN, MAI-KHANH            Midwest     10800.00                   21600.00   
                           Northeast   16000.00                   23300.00   
                           South       17850.00                   23250.00   
                           West        21800.00                   45850.00   

                                      same_state  
candidate_name             Region                 
CAFORIO, BRYAN             Midwest         False  
                           Northeast       False  
                           South           False  
                           West            False  
DELGADO, ANTONIO           Midwest         False  
                           Northeast       False  
                           South           F

### get summary stats for all house dems fundraising totals

In [44]:
indv_receipts_df.groupby("candidate_name").sum()["amount"].describe()

count       104.000000
mean     112920.687788
std      129579.747034
min          18.820000
25%       11445.480000
50%       63276.465000
75%      163301.717500
max      601470.000000
Name: amount, dtype: float64